# Federated Protein Embeddings and Task Model Fitting with BioNeMo

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This notebook will walk you through the task fitting workflow in the following sections:

* 
*
*

### Install requirements
Let's start by installing and importing library dependencies. We'll use requests to interact with the BioNeMo service, BioPython to parse FASTA sequences into SeqRecord objects, scikit-learn for classification tasks, and matplotlib for visualization.

In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, users can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [2]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [3]:
import io
import requests
from Bio import SeqIO

# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

Downloaded 13949 sequences


### Data splitting
Next, we prepare the data for simulating federated learning using `n_clients`.

In [4]:
n_clients = 3
n_samples_per_client = 100  # training samples
n_samples_per_client = 100000
# limiting to the proteins with sequence length<512 for embedding queries
MAX_SEQUENCE_LEN = 512
seed=0  # 0,1 works with 100
out_dir = "/tmp/fasta/mixed_soft"

import os
import re
import numpy as np
import pandas as pd
import uuid
import random
random.seed(seed)
        
# extract meta data and split
data = []
for i, x in enumerate(proteins):
        if len(str(x.seq)) > MAX_SEQUENCE_LEN:
            continue
            
        entry = {key: value for key, value in re.findall(r"([A-Z_]+)=(-?[A-z0-9]+[.0-9]*)", x.description)}
        entry["sequence"] = str(x.seq)
        entry["id"] = str(i)
       
        data.append(entry)
        
print(f"Read {len(data)} valid sequences.")
        
# shuffle & split training data
random.shuffle(data)
data_splits = np.array_split(data, n_clients)

# save split data
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

# save split training data
for idx, split in enumerate(data_splits):
    client_name = f"site-{idx+1}"
    
    split_dict = {}
    n_train = 0
    n_test = 0
    for entry in split:
        if entry["SET"] == "train":
            if n_train >= n_samples_per_client:
                continue            
            n_train += 1
        elif  entry["SET"] == "test":
            n_test += 1
        
        for k, v in entry.items():
            if k not in split_dict:
                split_dict[k] = []
            else:
                split_dict[k].append(v)

    df = pd.DataFrame(split_dict)
    df.to_csv(os.path.join(out_dir, f"data_{client_name}.csv"), index=False, columns=["id", "sequence", "TARGET", "SET"])    
    print(f"Saving {len(split_dict['id'])} sequences for client {client_name} ({n_train} train, {n_test} test).")

Read 8619 valid sequences.
Saving 2872 sequences for client site-1 (2310 train, 563 test).
Saving 2872 sequences for client site-2 (2276 train, 597 test).
Saving 2872 sequences for client site-3 (2333 train, 540 test).


### Federated embedding extraction
Running inference of the ESM-1nv model to extract embeddings requires a GPU with at least 12 GB memory. Here we run inference on each client sequentially using one thread to preserve GPU memory.

In [5]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/embeddings",
    workspace="/tmp/nvflare/bionemo/embeddings",
    n_clients=n_clients,
    threads=1  # due to memory constraints, we run the client execution sequentially in one thread
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-08-03 20:40:48,478 - SimulatorRunner - INFO - Create the Simulator Server.
2023-08-03 20:40:48,482 - Cell - INFO - server: creating listener on tcp://0:60867
2023-08-03 20:40:48,500 - Cell - INFO - server: created backbone external listener for tcp://0:60867
2023-08-03 20:40:48,502 - ConnectorManager - INFO - 255838: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-08-03 20:40:48,504 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:57479] is starting
2023-08-03 20:40:49,007 - Cell - INFO - server: created backbone internal listener for tcp://localhost:57479
2023-08-03 20:40:49,010 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:60867] is starting
2023-08-03 20:40:49,105 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 48003
2023-08-03 20:40:49,106 - SimulatorRunner - INFO - Deploy the Apps.
2023-08-03 20:40:49,912 - SimulatorRunner - INFO - Create the simula

[NeMo W 2023-08-03 20:41:05 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-03 20:41:06,616 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=b97172d0-daad-43c9-9247-9c71852ddd5c]: assigned task to client site-1: name=bionemo_inference, id=b97172d0-daad-43c9-9247-9c71852ddd5c


[NeMo W 2023-08-03 20:41:06 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-03 20:41:06,619 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=b97172d0-daad-43c9-9247-9c71852ddd5c]: sent task assignment to client. client_name:site-1 task_id:b97172d0-daad-43c9-9247-9c71852ddd5c
2023-08-03 20:41:06,622 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: bionemo_inference   task_id: b97172d0-daad-43c9-9247-9c71852ddd5c  sharable_header_task_id: b97172d0-daad-43c9-9247-9c71852ddd5c
[NeMo I 2023-08-03 20:41:06 utils:250] Restoring model from /tmp/nvflare/bionemo/embeddings/simulate_job/app_site-1/models/esm1nv.nemo
[NeMo I 2023-08-03 20:41:06 utils:254] Loading model class: bionemo.model.protein.esm1nv.esm1nv_model.ESM1nvModel
2023-08-03 20:41:06,731 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: True (cuda), used: True
2023-08-03 20:41:06,732 - pytorch_lightning.utilities.rank_zero - INFO - TPU ava

...

Predicting DataLoader 0: 100%|██████████| 23/23 [00:28<00:00,  1.25s/it]2023-08-03 20:43:21,957 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-3, peer_run=simulate_job]: got result from client site-3 for task: name=bionemo_inference, id=6d54558d-d10c-41c4-bb5d-666a9660f745
2023-08-03 20:43:21,961 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-3, peer_run=simulate_job, peer_rc=OK, task_name=bionemo_inference, task_id=6d54558d-d10c-41c4-bb5d-666a9660f745]: finished processing client result by bionemo_inference
2023-08-03 20:43:21,964 - SubmitUpdateCommand - INFO - submit_update process. client_name:site-3   task_id:6d54558d-d10c-41c4-bb5d-666a9660f745
2023-08-03 20:43:21,969 - SimulatorClientRunner - INFO - Simulate Run client: site-1 on GPU group: None
2023-08-03 20:43:22,132 - BioNeMoInference - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: task bi

[NeMo W 2023-08-03 20:43:34 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-03 20:43:35,536 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job]: server runner is finalizing - asked client to end the run
2023-08-03 20:43:35,538 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: __end_run__   task_id:   sharable_header_task_id: 
2023-08-03 20:43:35,546 - SimulatorClientRunner - INFO - Simulate Run client: site-2 on GPU group: None
2023-08-03 20:43:35,550 - FederatedClient - INFO - Shutting down client run: site-1
2023-08-03 20:43:35,550 - FederatedClient - INFO - Shutting down client run: site-2
2023-08-03 20:43:35,552 - FederatedClient - INFO - Shutting down client run: site-3
2023-08-03 20:43:35,553 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: asked to abort - triggered abort_signal to stop the RUN


[NeMo W 2023-08-03 20:43:35 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-03 20:43:38,925 - MPM - INFO - MPM: Good Bye!
Simulator finished with run_status 0


### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo server are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM-1nv, `D=768`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [6]:
# load embeddings from site-1
import pickle
protein_embeddings = pickle.load(open(os.path.join(out_dir, "data_site-1.pkl"), "rb"))
print(f"Loaded {len(protein_embeddings)} embeddings from site-1.")

for i in range(4):
    protein_embedding = protein_embeddings[i]
    print(f"Inference result contains {list(protein_embedding.keys())}")
    x = protein_embedding["embeddings"]
    print(f"{protein_embedding['id']}: range {np.min(x)}-{np.max(x)}, mean={np.mean(x)}, shape={x.shape}")

Loaded 2862 embeddings from site-1.
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
10649: range -0.90283203125-1.7978515625, mean=-0.0032165292650461197, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
7598: range -0.8017578125-1.2138671875, mean=-0.0010715207317844033, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
11506: range -0.86865234375-1.708984375, mean=-0.0034913886338472366, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
7116: range -0.7880859375-1.5556640625, mean=-0.0015334287891164422, shape=(768,)


Let's enumerate the labels corresponding to potential subcellular locations.

In [7]:
# Let's also print all the labels

labels = set([entry['TARGET'] for entry in data])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

1. Plastid
2. Mitochondrion
3. Nucleus
4. Peroxisome
5. Golgi apparatus
6. Cytoplasm
7. Extracellular
8. Endoplasmic reticulum
9. Cell membrane
10. Lysosome


### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier. The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the call below, we define the MLP to use the Adam optimizer with a network of 32 hidden layers, defining a random state (or seed) for reproducibility, and trained for a maximum of 500 iterations.

#### Local training

In [ ]:
os.environ["SIM_LOCAL"] = "True"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace="/tmp/nvflare/bionemo/local",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-08-04 18:23:13,206 - SimulatorRunner - INFO - Create the Simulator Server.
2023-08-04 18:23:13,211 - Cell - INFO - server: creating listener on tcp://0:44259
2023-08-04 18:23:13,231 - Cell - INFO - server: created backbone external listener for tcp://0:44259
2023-08-04 18:23:13,232 - ConnectorManager - INFO - 329628: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-08-04 18:23:13,237 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:60664] is starting
2023-08-04 18:23:13,741 - Cell - INFO - server: created backbone internal listener for tcp://localhost:60664
2023-08-04 18:23:13,744 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:44259] is starting
2023-08-04 18:23:13,838 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 48351
2023-08-04 18:23:13,840 - SimulatorRunner - INFO - Deploy the Apps.
2023-08-04 18:23:13,847 - SimulatorRunner - INFO - Create the simula

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


2023-08-04 18:23:17,411 - BioNeMoMLPModelPersistor - INFO - [identity=simulator_server, run=simulate_job]: MLPClassifier coefficients [(768, 128), (128, 64), (64, 10)], intercepts [(128,), (64,), (10,)]
2023-08-04 18:23:17,414 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: starting workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) ...
2023-08-04 18:23:17,421 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job]: Initializing ScatterAndGather workflow.
2023-08-04 18:23:17,430 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: Workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) started
2023-08-04 18:23:17,438 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Beginning ScatterAndGather training phase.
2023-08-04 18:23:17,447 - ScatterAndGather - INFO - [identity=simulator_

#### Federated learning

In [ ]:
import os
os.environ["SIM_LOCAL"] = "False"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace="/tmp/nvflare/bionemo/fedavg",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)